# Evaluation Additional Modeling Pipelines
We should also compare performance in the evaluation data with other readily available phonetic transcription options, to determine whether fine-tuning your own model is worth the effort. 
The two options we consider here are: 
- Wav2vec2 fine tuned on TIMIT (https://huggingface.co/elgeish/wav2vec2-large-lv60-timit-asr) as the speech recognition model, followed by using [epitran](https://github.com/dmort27/epitran) to convert othography to IPA. The TIMIT corpus is a high quality corpus of read English speech.
- [Allosaurus](https://github.com/xinjli/allosaurus) is a pre-trained universal phone recognizer that claims to recognize phones in more than 2000 languages. 
- [Whisper](https://openai.com/index/whisper/) is the state-of-the-art sequence-to-sequence speech recognition model released by OpenAI. Details about the different model releases are available at https://github.com/openai/whisper/blob/main/model-card.md. There are multilingual and English fine-tuned versions.

These evaluations only need to be run and computed once. 

## Additional installation step for Epitran
To use Epitran for English, you also need to install https://github.com/festvox/flite. See the Epitran note at https://github.com/dmort27/epitran?tab=readme-ov-file#installation-of-flite-for-english-g2p.  I installed Flite on my mac:

```bash
$ git clone http://github.com/festvox/flite
$ cd flite
$ ./configure && make
$ sudo make install
$ cd testsuite
$ make lex_lookup
$ sudo cp lex_lookup /usr/local/bin
```



In [1]:
import itertools
import time
from pathlib import Path

import allosaurus.app
import allosaurus.bin.download_model
import datasets
import epitran
import transformers
from tqdm import tqdm


from multipa.data_utils import load_buckeye_split, clean_text
from multipa.evaluation import ModelEvaluator, preprocess_test_data, write_detailed_prediction_results, DETAILED_PREDICTIONS_CSV_SUFFIX, PREDICTION_KEY

VERBOSE_RESULTS_DIR = Path("../data/evaluation_results/detailed_predictions")
AGGREGATE_METRICS_CSV = Path("../data/evaluation_results/aggregate_metrics/epitran_allosaurus_eval.csv")
EDIT_DIST_DIR = Path("../data/evaluation_results/edit_distances/")

IS_REMOVE_SPACES = True
NUM_PROC = 8 # For HuggingFace dataset map and filter

/Users/virginia/miniconda3/envs/multipa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def allosaurus_predict(test_dataset, model="eng2102", phone_inventory="ipa"):
    print("Evaluating allosaurus. Model:", model, "Phone inventory:", phone_inventory)
    model_predictions = []
    model = allosaurus.app.read_recognizer(model)
    start = time.time()
    for audio in tqdm(test_dataset["audio"]):
        prediction = model.recognize(audio["path"], phone_inventory)
        prediction = prediction.replace(" ", "")
        model_predictions.append({PREDICTION_KEY: prediction})
    end = time.time()
    print("Eval time in seconds:", end-start)
    predictions_dataset = datasets.Dataset.from_list(model_predictions)
    predictions_dataset = predictions_dataset.map(
        lambda x: clean_text(x, text_key=PREDICTION_KEY, is_remove_space=IS_REMOVE_SPACES), num_proc=NUM_PROC
    )
    return predictions_dataset

def hf_model_to_epitran_predict(model_name, test_dataset):
    print("Building pipeline and downloading model")
    pipe = transformers.pipeline("automatic-speech-recognition", model=model_name, generate_kwargs={"language": "english"})
    print("Predicting with", model_name)
    start = time.time()
    orthography_predictions = [d["text"] for d in pipe(test_dataset["audio"])]
    epi = epitran.Epitran('eng-Latn')
    print("Transliterating with Epitran")
    ipa_predictions = []
    for pred in tqdm(orthography_predictions):
        result = epi.transliterate(pred).replace(" ", "")
        ipa_predictions.append({PREDICTION_KEY: result})
    end = time.time()
    print("Eval time in seconds:", end-start)
    predictions_dataset = datasets.Dataset.from_list(ipa_predictions)
    predictions_dataset = predictions_dataset.map(
        lambda x: clean_text(x, text_key=PREDICTION_KEY, is_remove_space=IS_REMOVE_SPACES), num_proc=NUM_PROC
    )
    return predictions_dataset

In [3]:
input_data = load_buckeye_split("../data/buckeye", "test")
# Snippet of transcriptions
# Note that there don't appear to be any non-empty transcriptions,
# so this notebook skips looking at hallucinations
print("Data Preview")
print(input_data)
print(input_data[0])

non_empty_test_data, empty_test_data = preprocess_test_data(input_data, is_remove_space=True)

print("Test data with speech transcriptions")
print(non_empty_test_data)
print(non_empty_test_data[0])
print("Test data without speech")
print(empty_test_data)

model_evaluator = ModelEvaluator()

Data Preview
Dataset({
    features: ['utterance_id', 'duration', 'buckeye_transcript', 'text', 'ipa', 'speaker_id', 'speaker_gender', 'speaker_age_range', 'interviewer_gender', 'file_path', 'audio', '__index_level_0__'],
    num_rows: 5079
})
{'utterance_id': 's2501a_Utt0', 'duration': 0.925981, 'buckeye_transcript': 'f ao r f ay v', 'text': 'four five', 'ipa': 'f ɔ ɹ f aɪ v', 'speaker_id': 'S25', 'speaker_gender': 'f', 'speaker_age_range': 'o', 'interviewer_gender': 'm', 'file_path': 'data/buckeye/test/s2501a_Utt0.wav', 'audio': {'bytes': None, 'path': '/Users/virginia/workspace/multipa/data/buckeye/test/s2501a_Utt0.wav'}, '__index_level_0__': 0}


Filter: 100%|██████████| 5079/5079 [00:01<00:00, 3527.76 examples/s]


Test data with speech transcriptions
Dataset({
    features: ['utterance_id', 'duration', 'buckeye_transcript', 'text', 'ipa', 'speaker_id', 'speaker_gender', 'speaker_age_range', 'interviewer_gender', 'file_path', 'audio', '__index_level_0__'],
    num_rows: 5079
})
{'utterance_id': 's2501a_Utt0', 'duration': 0.925981, 'buckeye_transcript': 'f ao r f ay v', 'text': 'four five', 'ipa': 'fɔɹfaɪv', 'speaker_id': 'S25', 'speaker_gender': 'f', 'speaker_age_range': 'o', 'interviewer_gender': 'm', 'file_path': 'data/buckeye/test/s2501a_Utt0.wav', 'audio': {'path': '/Users/virginia/workspace/multipa/data/buckeye/test/s2501a_Utt0.wav', 'array': array([-0.00997925, -0.01052856, -0.00958252, ...,  0.00085449,
        0.00061035,  0.00042725]), 'sampling_rate': 16000}, '__index_level_0__': 0}
Test data without speech
Dataset({
    features: ['utterance_id', 'duration', 'buckeye_transcript', 'text', 'ipa', 'speaker_id', 'speaker_gender', 'speaker_age_range', 'interviewer_gender', 'file_path', 'aud

In [4]:
models = [
    "openai/whisper-large-v3-turbo",
    "openai/whisper-large-v3",
    "openai/whisper-medium.en",
]
for m in models:
    # Epitran
    epitran_predictions = hf_model_to_epitran_predict(m, non_empty_test_data)
    model_name = f"{m}_to_epitran".replace("/", "_")
    metrics = model_evaluator.eval_non_empty_transcriptions(
        model_name, epitran_predictions[PREDICTION_KEY], non_empty_test_data["ipa"]
    )
    write_detailed_prediction_results(VERBOSE_RESULTS_DIR, model_name, non_empty_test_data, epitran_predictions, metrics)
    model_evaluator.write_edit_distance_results(model_name,EDIT_DIST_DIR)

Building pipeline and downloading model


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Predicting with openai/whisper-large-v3-turbo


/Users/virginia/miniconda3/envs/multipa/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:483: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=english, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=english.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


KeyboardInterrupt: 

In [ ]:
# Define models and phone inventory to test
allosaurus_models = ["uni2005", "eng2102"]
phone_inventory = ["ipa", "eng"]

# Download models
for m in allosaurus_models:
    allosaurus.bin.download_model.download_model(m)

# Predict and check against gold standard
for model, pi in itertools.product(allosaurus_models, phone_inventory):
    model_predictions = allosaurus_predict(non_empty_test_data, model, pi)
    model_name = f"allosaurus_{model}_{pi}"
    detailed_results_csv = f"{model_name}_{DETAILED_PREDICTIONS_CSV_SUFFIX}"
    metrics = model_evaluator.eval_non_empty_transcriptions(model_name, model_predictions, non_empty_test_data["ipa"])
    write_detailed_prediction_results(
        VERBOSE_RESULTS_DIR, model_name, non_empty_test_data, model_predictions, metrics
    )
    model_evaluator.write_edit_distance_results(model_name, EDIT_DIST_DIR)


In [ ]:
# Write all results to file for comparison
model_evaluator.to_csv(AGGREGATE_METRICS_CSV)